In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as mpl
from datetime import timedelta

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from generate_results_utils import *
from datetime import timedelta, date, datetime

# colors = ["#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
#           "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000"]
colors = ['#000000', '#0080ff', '#009900', "#ff9933", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
          "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000", "#e6194b", "#3cb44b", "#ffe119", ]


start_train = datetime.strptime("22.06.2022", "%d.%m.%Y") 
end_train = start_train + timedelta(days=27) # , hours=23, minutes=59, seconds=59)
split_dates = []
for i in range(5):
    split_dates.append(end_train)
    end_train += timedelta(days=28)

In [ ]:
start_train = datetime.strptime("22.06.2022", "%d.%m.%Y") 
end_train = start_train + timedelta(days=14) # , hours=23, minutes=59, seconds=59)
period_labels = []
for i in range(6):
    period_labels.append(end_train)
    end_train += timedelta(days=28)

In [ ]:
actions_suffix = '_actions.csv'
value_suffix = '_state.csv'

In [ ]:
def get_dfs(stat_files):
    dfs = []
    for fname in stat_files:
        portfolio_prefix = f'../results/ce/{fname}/all'

        df = pd.read_csv(f"{portfolio_prefix}{value_suffix}")  # , index_col=0
        df = df.rename(columns={df.columns[0]: 'date'})
        df.loc[0, 'date'] = pd.to_datetime(df.loc[1]['date']) - timedelta(days=1)
        df['date'] = pd.to_datetime(df['date'])
        df['return'] = df['account_value'].pct_change() + 1
        df['equity'] = (df['return']).cumprod()
        df.loc[0, 'return'] = 1
        df.loc[0, 'equity'] = 1
        df.index = df['date']
        df = df.drop('date', axis=1)

        # df2 = pd.read_csv(f"{portfolio_prefix}{actions_suffix}", index_col=0)
        # df2.index = pd.to_datetime(df2.index)

        df['strategy'] = df['return']
        df_strategy = df[['strategy']]         
        df_returns = df_strategy.cumprod()
        dfs.append(df_returns)
    return dfs

def get_benchmark(dfs, bah_only=False):
    benchmark_returns_1d = '../datasets/thesis/benchmark/1d_benchmark_returns.csv'
    df_benchmark = pd.read_csv(f"{benchmark_returns_1d}")
    df_benchmark.index = pd.to_datetime(df_benchmark['date'])
    df_benchmark = df_benchmark.drop('date', axis=1)
    if bah_only:
        df_benchmark = df_benchmark.drop('MPT', axis=1)
        df_benchmark = df_benchmark.drop('CRP', axis=1)
    df_benchmark = df_benchmark[(df_benchmark.index >= dfs[0].index[0]) & (df_benchmark.index <= dfs[0].index[-1])]
    df_benchmark = df_benchmark.cumprod()
    return df_benchmark

def get_prices(sample_strat):
    stock_prices = '../datasets/thesis/1h/all_1h.csv'
    df3 = pd.read_csv(stock_prices)
    df3['date'] = pd.to_datetime(df3['date'])
    # get or column with close price for each stock per tick
    df3 = df3.pivot(index="date", columns="tic", values="close")
    # limit dataset to portfolio range
    df3 = df3[(df3.index >= sample_strat.index[0]) & (df3.index <= sample_strat.index[-1])]
    # get relative change from price
    df3 = df3.pct_change()
    # set first row to 0
    df3.loc[df3.iloc[0].name] = 0
    # get cumulative return for each stock
    df3 = (df3 + 1).cumprod()
    return df3

def plot_performance(title, names, strats, benchmarks, prices, plot_folder, show_prices=True, save=True, show_title=True, split_dates=[], winner=[], text_pos=[], text_y=1):    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if show_prices:
        for idx, name in enumerate(prices.columns):
            fig.add_trace(go.Scatter(x=prices.index,y=prices[name],name=name, marker={'color':'#fff'}),secondary_y=False)

    for idx, name in enumerate(names):
        cdf = strats[idx]
        fig.add_trace(go.Scatter(x=cdf.index,y=cdf['strategy'],name=name, marker_color=colors[idx]),secondary_y=False)

    # benchmarks = dfs[0][["CRP", "BAH", "MPT"]]
    cs = ['#ff0000', '#36454F', '#A9A9A9', '#8A9A5B']
    for idx, name in enumerate(benchmarks.columns):
        fig.add_trace(go.Scatter(x=benchmarks.index,y=benchmarks[name],name=name, marker={'color':cs[0]}, line={'dash':'dashdot'}),secondary_y=False)
        
    for i, split in enumerate(split_dates):
        fig.add_vline(x=split, line_width=2, line_color='#ccc', opacity=0.7)
        fig.update_xaxes(showgrid=False)
    
    if len(winner) > 0:
        fig.add_trace(go.Scatter(x=text_pos, y=[text_y] * len(winner), mode="text", text=winner, showlegend=False, textposition="bottom center"))
    
    if show_title:
        fig.update_layout(height=400, template='plotly_white') # title_text=f'{title} Strategy Performance compared to benchmarks', 
    else:
        fig.update_layout(height=600)
    fig.show()
    if save:
        filename = f"{plot_folder}/performance_compare_{title.replace(' ', '_')}"
        fig.write_html(f"{filename}.html")
        fig.write_image(f"{filename}.png", width=1200, scale=2)  

# Performance

In [ ]:
[1] * 5

In [ ]:
names = ["Ensemble (1d)"]
stat_files = ["ce_1d_2301221639_fix"]  #"ce_1d_2301132037_1", "ce_1d_2301141639_1", "ce_1d_2301141954_1", "ce_1d_2301150052_6x", 

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, bah_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "1D Ensemble"

portfolio_prefix = f'../results/ce/{stat_files[0]}/all'
winner = pd.read_csv(f"{portfolio_prefix}_stats.csv", index_col=0)['winner']

plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False, split_dates=split_dates, winner=winner, text_pos=period_labels, text_y=1.5)

In [ ]:
names = ["Ensemble (12h)"]
stat_files = ["ce_12h_2301132144_1"] # , "ce_12h_2301142119_1", "ce_12h_2301142318_1"

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, bah_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "12H Ensemble"

portfolio_prefix = f'../results/ce/{stat_files[0]}/all'
winner = pd.read_csv(f"{portfolio_prefix}_stats.csv", index_col=0)['winner']

plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False, split_dates=split_dates, winner=winner, text_pos=period_labels, text_y=2.1)

In [ ]:
names = ["Ensemble (6h)"]
stat_files = ["ce_6h_2301132259_1"]

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, bah_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "6H Ensemble"

portfolio_prefix = f'../results/ce/{stat_files[0]}/all'
winner = pd.read_csv(f"{portfolio_prefix}_stats.csv", index_col=0)['winner']

plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False, split_dates=split_dates, winner=winner, text_pos=period_labels, text_y=1.7)

In [ ]:
names = ["Ensemble (1h)"]
stat_files = ["ce_1h_2301141443_1"]

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, bah_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "1H Ensemble"

portfolio_prefix = f'../results/ce/{stat_files[0]}/all'
winner = pd.read_csv(f"{portfolio_prefix}_stats.csv", index_col=0)['winner']

plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False, split_dates=split_dates, winner=winner, text_pos=period_labels, text_y=1.5)

# Weights

In [ ]:
def get_weights(run_dir, run_name, res, root_dir='..', file_prefix='all'):
    results_dir = f"{root_dir}/results/{run_dir}/{run_name}"
    results_file_prefix = f"{results_dir}/{file_prefix}"
    filename = f"{results_file_prefix}_state.csv"
    
    results_account_value = pd.read_csv(filename)
    ratios = get_asset_ratios(results_account_value, res)
    ratio_values = convert_ratios_to_value(ratios, results_account_value)
    
    return ratios, ratio_values

def generate_weights_for_run(run_dir, run_name, res, root_dir='..', file_prefix='all', save=True, show=False, show_both=False):
    
    ratios, ratios_full = get_weights(run_dir, run_name, res, root_dir, file_prefix)    
    
    if save:
        weights_norm_file = f"{results_dir}/weights_normalized.png"
        weights_file = f"{results_dir}/weights.png"
    else:
        weights_norm_file, weights_file = "", ""
    
    print_weights(ratios, save_location=weights_norm_file, show=(show and show_both))
    print_weights(ratios_full, save_location=weights_file, show=show)
    
def generate_weights_plot(ratio, names, colors, filename_prefix, show=False, split_dates=[]):
    fig = go.Figure()
    x = ratio['date']
    for idx, name in enumerate(names):
        y = ratio[name]
        # fill = 'tonexty' if idx > 0 else 'tozeroy'
        fig.add_trace(go.Scatter(x=x, y=y, hoverinfo='x+y', mode='lines', name=name.replace('USDT', ''), line=dict(width=0.5, color=colors[idx]), stackgroup='one', legendrank=10-idx))
    fig.update_layout(height=400, template='plotly_white') # title_text=f'Weights', 
    for i, split in enumerate(split_dates):
        fig.add_vline(x=split, line_width=2, line_color='#ccc', opacity=0.7)
        fig.update_xaxes(showgrid=False)
    if show:
        fig.show()
    if filename_prefix != "":
        fig.write_html(f"{filename_prefix}/weights.html")
        fig.write_image(f"{filename_prefix}/weights.png", width=1200, scale=1) 

In [ ]:
alg = 'A2C'
res_list = ["1h", "6h", "12h", "1d"]
runs = ["ce_1h_2301141443_1", "ce_6h_2301132259_1", "ce_12h_2301132144_1", "ce_1d_2301221639_fix"]

for idx, res in enumerate(res_list):
    run = runs[idx]
    print(run, res)    
    ratios, ratio_values = get_weights('ce', f'{run}', res)
    generate_weights_plot(ratio_values, crypto_names, weight_colors, f"../results/ce/{run}", split_dates=split_dates)